In [2]:
# import org.apache.spark.sql.cassandra._

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.streaming import StreamingContext
from Activity import Activity
from activityByProduct import activityByProduct
from pyspark.streaming.kafka import KafkaUtils
import os

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.ui.port=4040 --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.0,com.datastax.spark:spark-cassandra-connector_2.11:2.0.0-M3 pyspark-shell'

In [3]:
conf = SparkConf() \
    .setAppName("Streaming test") \
    .setMaster("local[2]") \
    .set("spark.cassandra.connection.host", "127.0.0.1")
sc = SparkContext(conf=conf) 
sqlcontext=SQLContext(sc)

In [4]:
# conf.setAppName("Spark Cassandra")

In [5]:
# conf.set("spark.cassandra.connection.host","localhost")

In [6]:
# # sc = SparkContext(appName="PythonSparkStreamingKafka")
# sc = SparkContext(conf=conf)

# sc.setLogLevel("WARN")

In [7]:
# sqlcontext = SQLContext(sc)

In [8]:
# from cassandra.cluster import Cluster

In [9]:
# cluster = Cluster()

In [10]:
# session = cluster.connect('lambda')

In [11]:
# rows = session.execute("SELECT * FROM lambda.emp;")

In [12]:
# rows.column_types

In [13]:
ssc = StreamingContext(sc,2)

In [14]:
textDStream1 = KafkaUtils.createStream(ssc, 'localhost:2181', 'spark-streaming', {'data_pipeline':1})
textDStream = textDStream1.map(lambda x: x[1])

In [15]:
def subprocess(line):
    records = line.split("\t")
    if records[0] != "action" and len(records) == 7:
        return Activity(records[0],records[1],records[2],records[3],records[4],records[5],records[6])
    else:
        return None

In [16]:
def process(rdd):
    if not rdd.isEmpty():
        newRDD = rdd.map(subprocess)
    else:
        newRDD = None
    return newRDD

In [17]:
def process_query(rdd):
    if not rdd.isEmpty():
        newDF=rdd.map(lambda x: (x, )).toDF()
        newDF2 = newDF.select('_1.*')
        newDF2.createOrReplaceTempView("Activity")
#         toHDFS = sqlcontext.sql("select * from activity")
# #         toHDFS.write.mode("append").csv("hdfs://localhost:9000/kafka_spark",header=True)
#         toHDFS2 = toHDFS.select(['action', 'page', 'product', 'referrer', 'timestamp_hour', 'visitor'])
        visitorsByProduct = sqlcontext.sql("SELECT product, timestamp_hour, COUNT(DISTINCT visitor) as unique_visitors FROM Activity GROUP BY product, timestamp_hour")
        visitorsByProduct.show(10)
        visitorsByProduct.printSchema()
#         newDF2.show(10)
        visitorsByProduct.write.format("org.apache.spark.sql.cassandra").mode('append').options(table="stream_visitors_by_product", keyspace="lambda").save()

In [18]:
newDStream = textDStream.transform(process)

In [19]:
newDStream.foreachRDD(process_query)

In [22]:
# newDStream.pprint()

In [20]:
ssc.start()

+--------------------+--------------+---------------+
|             product|timestamp_hour|unique_visitors|
+--------------------+--------------+---------------+
|Harney & Sons,Spi...| 1521808397116|              1|
|L'Oreal,Out of Be...| 1521808390282|              1|
|Simple,Eye Makeup...| 1521808401673|              1|
|Hot Shot,Flying I...| 1521808391593|              1|
|Tide,Febreze Fres...| 1521808392202|              1|
|Crest,Pro-Health ...| 1521808397713|              1|
|  Charmin,Ultra Soft| 1521808398793|              1|
|Siggi's,"Icelandi...| 1521808392202|              1|
|Comet,Comet With ...| 1521808398793|              1|
|Crisco,Pure Veget...| 1521808385116|              2|
+--------------------+--------------+---------------+
only showing top 10 rows

root
 |-- product: string (nullable = true)
 |-- timestamp_hour: string (nullable = true)
 |-- unique_visitors: long (nullable = false)

+--------------------+--------------+---------------+
|             product|times

+--------------------+--------------+---------------+
|             product|timestamp_hour|unique_visitors|
+--------------------+--------------+---------------+
|         OYO,Whiskey| 1521808404590|              1|
|   CVS,2-Ply Tissues| 1521808419470|              1|
|      Kroger,2% Milk| 1521808419470|              1|
| Ralphs,Plastic Wrap| 1521808408910|              1|
|Sharpie,Fine Perm...| 1521808409390|              1|
|Expo,Dry Erase Ma...| 1521808407950|              1|
|Kirkland Signatur...| 1521808404590|              1|
|Sensodyne,Fresh I...| 1521808405070|              1|
|Crest,Pro-Health ...| 1521808414190|              1|
|Alessi,Garlic Bre...| 1521808418510|              1|
+--------------------+--------------+---------------+
only showing top 10 rows

root
 |-- product: string (nullable = true)
 |-- timestamp_hour: string (nullable = true)
 |-- unique_visitors: long (nullable = false)

+--------------------+--------------+---------------+
|             product|times

+--------------------+--------------+---------------+
|             product|timestamp_hour|unique_visitors|
+--------------------+--------------+---------------+
|Edge,Sensitive Sk...| 1521808433957|              1|
|CVS Total Home,"A...| 1521808432517|              1|
|      Coca Cola,Soda| 1521808433957|              1|
|Dust Destroyer,Co...| 1521808435397|              1|
|Banana Boat,Sport...| 1521808425797|              1|
|Disney,Tangled (DVD)| 1521808434917|              1|
|Fernet-Branca,Liq...| 1521808435397|              1|
|Banquet,Brown 'N ...| 1521808432997|              1|
|De Cecco,Penne Ri...| 1521808426757|              1|
|Colgate,Toothbrus...| 1521808429157|              1|
+--------------------+--------------+---------------+
only showing top 10 rows

root
 |-- product: string (nullable = true)
 |-- timestamp_hour: string (nullable = true)
 |-- unique_visitors: long (nullable = false)

+--------------------+--------------+---------------+
|             product|times

+--------------------+--------------+---------------+
|             product|timestamp_hour|unique_visitors|
+--------------------+--------------+---------------+
|Trader Joe's,Curr...| 1521808447927|              1|
|Blue Diamond,Almo...| 1521808447927|              1|
|Scrabble,Crosswor...| 1521808445047|              1|
|Neutrogena,On-the...| 1521808449367|              1|
|Kroger,Vanilla Ex...| 1521808455127|              1|
|Trident,"White Gu...| 1521808444087|              1|
|Grandma's Seasoni...| 1521808445527|              1|
|Biore,Deep Pore C...| 1521808442167|              1|
|Avery,Heavy-Duty ...| 1521808444567|              1|
|Simple Truth Orga...| 1521808445047|              1|
+--------------------+--------------+---------------+
only showing top 10 rows

root
 |-- product: string (nullable = true)
 |-- timestamp_hour: string (nullable = true)
 |-- unique_visitors: long (nullable = false)

+--------------------+--------------+---------------+
|             product|times

+--------------------+--------------+---------------+
|             product|timestamp_hour|unique_visitors|
+--------------------+--------------+---------------+
|Crest,Pro-Health ...| 1521808462417|              1|
|Siggi's,"Icelandi...| 1521808462897|              1|
|Nature Made,D3 40...| 1521808463857|              1|
|Redbreast,Single ...| 1521808465297|              1|
|Biotene,Dry Mouth...| 1521808470097|              1|
|Glad,Odor Shield ...| 1521808459537|              1|
|  Charmin,Ultra Soft| 1521808462417|              1|
|Clover Stornetta,...| 1521808467697|              1|
|   Tapatio,Hot Sauce| 1521808467697|              1|
|Westvleteren,West...| 1521808465777|              1|
+--------------------+--------------+---------------+
only showing top 10 rows

root
 |-- product: string (nullable = true)
 |-- timestamp_hour: string (nullable = true)
 |-- unique_visitors: long (nullable = false)

+--------------------+--------------+---------------+
|             product|times

+--------------------+--------------+---------------+
|             product|timestamp_hour|unique_visitors|
+--------------------+--------------+---------------+
|Tapatio,Salsa Pic...| 1521808486937|              1|
|CVS Pharmacy,200m...| 1521808490297|              1|
|Band-Aid,Flexible...| 1521808477817|              1|
|Kettle,Classic Ba...| 1521808487417|              1|
|  Bolla,Pinot Grigio| 1521808478777|              1|
|Reynolds,Parchmen...| 1521808487417|              1|
|  Dixie,Paper Plates| 1521808482137|              1|
|Sol Republic,JAX ...| 1521808478297|              1|
|SteamFast,Travel ...| 1521808483577|              1|
|L'Oreal,Out of Be...| 1521808478777|              1|
+--------------------+--------------+---------------+
only showing top 10 rows

root
 |-- product: string (nullable = true)
 |-- timestamp_hour: string (nullable = true)
 |-- unique_visitors: long (nullable = false)

+--------------------+--------------+---------------+
|             product|times

In [ ]:
# ssc.stop(stopSparkContext=False,stopGraceFully=True)

+--------------------+--------------+---------------+
|             product|timestamp_hour|unique_visitors|
+--------------------+--------------+---------------+
|Kleenex,White Tis...| 1521808191860|              1|
|David's Tea,Choco...| 1521808180820|              1|
|Crest,Pro-Health ...| 1521808192340|              1|
|CVS Pharmacy,91% ...| 1521808192820|              1|
|Duracell,76A Dura...| 1521808192340|              1|
|Penn,Extra Duty F...| 1521808193300|              1|
|Mrs. Meyer's Clea...| 1521808185140|              1|
|Glass Plus,Glass ...| 1521808190900|              1|
|Avery,Permanent R...| 1521808191860|              1|
|Crest,Pro-Health ...| 1521808188500|              1|
+--------------------+--------------+---------------+
only showing top 10 rows

root
 |-- product: string (nullable = true)
 |-- timestamp_hour: string (nullable = true)
 |-- unique_visitors: long (nullable = false)

+--------------------+--------------+---------------+
|             product|times

In [25]:
# df = sqlcontext.read.csv('hdfs://localhost:9000/kafka_spark/part-00000-00535f69-81ae-4191-bda1-fbcd66209aaf-c000.csv', header=True, inferSchema=True)

In [26]:
# df.printSchema()

+--------------------+--------------+---------------+
|             product|timestamp_hour|unique_visitors|
+--------------------+--------------+---------------+
|Band-Aid,Flexible...| 1521801897075|              1|
|Kirkland Signatur...| 1521801905493|              1|
|Grandma's Seasoni...| 1521801924024|              1|
|Parallax Inc.,Ext...| 1521801915092|              1|
|Pereg,Organic Quinoa| 1521801904889|              1|
|Trident,"16ct Whi...| 1521801885761|              1|
|Ralphs,Nice 'n El...| 1521801906195|              1|
|Kroger,Dry Roaste...| 1521801937134|              1|
|CVS Total Home,"A...| 1521801890703|              1|
|Natural Grooming ...| 1521801885492|              1|
+--------------------+--------------+---------------+
only showing top 10 rows

root
 |-- product: string (nullable = true)
 |-- timestamp_hour: string (nullable = true)
 |-- unique_visitors: long (nullable = false)

+--------------------+--------------+---------------+
|             product|times

+--------------------+--------------+---------------+
|             product|timestamp_hour|unique_visitors|
+--------------------+--------------+---------------+
|Disney,Tangled (DVD)| 1521808175218|              1|
|Heinz,Distilled W...| 1521808170898|              1|
|Clover Stornetta,...| 1521808161778|              1|
|Crest,Pro-Health ...| 1521808174258|              1|
|Scrub Daddy,Spong...| 1521808159378|              1|
|Alessi,Garlic Bre...| 1521808179538|              1|
|Coop,Tiefkühlbeut...| 1521808174738|              1|
|Expo,Dry Erase Ma...| 1521808173778|              1|
|The Body Shop,Coc...| 1521808172818|              1|
|Staples,Standard ...| 1521808167058|              1|
+--------------------+--------------+---------------+
only showing top 10 rows

root
 |-- product: string (nullable = true)
 |-- timestamp_hour: string (nullable = true)
 |-- unique_visitors: long (nullable = false)

+--------------------+--------------+---------------+
|             product|times